## Projeto AceleraDev Data Science
## Objetivo

O objetivo deste produto é fornecer um serviço automatizado que recomenda leads para um usuário dado sua atual lista de clientes (Portfólio).

## Contextualização
Algumas empresas gostariam de saber quem são as demais empresas em um determinado mercado (população) que tem maior probabilidade se tornarem seus próximos clientes. Ou seja, a sua solução deve encontrar no mercado quem são os leads mais aderentes dado as características dos clientes presentes no portfólio do usuário.

Além disso, sua solução deve ser agnóstica ao usuário. Qualquer usuário com uma lista de clientes que queira explorar esse mercado pode extrair valor do serviço.


### Bases de Dados:

   - Mercado: Base com informações sobre as empresas do Mercado a ser considerado.
   - Portfolio 1: Ids dos clientes da empresa 1
   - Portfolio 2: Ids dos clientes da empresa 2
   - Portfolio 3 : Ids dos clientes da empresa 3
As bases de portfólio poderão ser utilizadas para testar a aderência da solução. Além disso, se a equipe desejar, poderá simular portfólios por meio de amostragens no mercado.

**As bases de portfólio poderão ser utilizadas para testar a aderência da solução. Além disso, se a equipe desejar, poderá simular portfólios por meio de amostragens no mercado.**

### Requisitos técnicos obrigatórios

- Utilizar técnicas de data science e machine learning para desenvolver o projeto;

- Apresentar o desenvolvimento e outputs do modelo em um Jupyter Notebook ou outra tecnologia de apresentação de Output de modelos de Machine Learning;

- A análise deve considerar os seguintes pontos: análise exploratória dos dados, tratamento dos dados, avaliação de algoritmos, treinamento do modelo, avaliação de performance do modelo e visualização dos resultados;

- Para a apresentação do projeto, o tempo entre o treinamento do modelo e o output deve ser menor que 20 min.


#### Para dar início ao desafio - Carregamentos das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\geyon\AppData\Roaming\Python\Python37\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#### Carregando a base de dados: Mercado
#### Avaliando as primeiras 5 linhas e o tamanho da tabela

In [2]:
market = pd.read_csv('estaticos_market.csv')

In [3]:
market= market.drop('Unnamed: 0', axis=1)

In [4]:
market.head()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,...,43.738462,93.266667,19.166667,26.0,26.0,27.0,0.0,-3.703704,0.0,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVICOS DIVERSOS,SERVIÇO,6.512329,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,3.200000,1 a 5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


Removendo colunas com valores faltantes acima de 40%

In [5]:
null_numbers, null_index = market.isnull().sum(), market.isnull().sum().index
null_tuple = tuple(zip(null_index,null_numbers))

In [6]:
var_removal = []
for null_index,null_numbers in null_tuple:
    if null_numbers > (market.shape[0] * 0.4):
        var_removal.append(null_index)
    else:
        pass

In [7]:
market = market.drop(var_removal, axis=1)

In [8]:
market.head(2)

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,...,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,...,0.0,44.0,47.0,41.0,2.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",3132172.8,3132172.8,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,False,...,0.0,27.0,27.0,27.0,1.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0,0


In [9]:
#Removendo colunas que parecem ter pouca relevância ou/e pouca variação

In [10]:
remover = ['fl_matriz',#removi pois não tem importância o CNPJ corresponder ou não
           'setor',#Coluna com pouca/nenhuma variabilidade
           'qt_socios_pj',
           'idade_media_socios',
           'idade_maxima_socios',#variaveis com relação as caracteristicas dos sócios, não devem apresentar relevância
           'idade_minima_socios',
           'qt_socios_st_regular',
           'dt_situacao','fl_matriz',
           'de_natureza_juridica',
           'idade_empresa_anos', 'fl_rm',#caracteristicas fiscais da empresa, aparentemente não fazem sentido
           'fl_me', 'fl_sa', 'fl_epp', 'fl_mei', 
           'fl_ltda', 'dt_situacao', 'fl_st_especial','fl_email',
           'fl_spa','fl_antt', 'fl_veiculo', 'fl_optante_simples',
           'vl_total_veiculos_pesados_grupo','fl_telefone',
           'vl_total_veiculos_leves_grupo', 'fl_optante_simei',
           'fl_simples_irregular','fl_passivel_iss',
           'qt_socios','qt_socios_pf','vl_faturamento_estimado_grupo_aux',
           'empsetorcensitariofaixarendapopulacao']

Avaliando os valores faltantes categóricos e não categóricos

In [11]:
market =market.drop(remover,axis=1)

In [12]:
market.select_dtypes(include='object').isnull().sum()

id                                         0
sg_uf                                      0
natureza_juridica_macro                    0
de_ramo                                    0
idade_emp_cat                              0
nm_divisao                              1927
nm_segmento                             1927
sg_uf_matriz                            1939
de_saude_tributaria                    14851
de_saude_rescencia                     14851
de_nivel_atividade                     11168
nm_meso_regiao                         58698
nm_micro_regiao                        58698
de_faixa_faturamento_estimado          27513
de_faixa_faturamento_estimado_grupo    27513
dtype: int64

In [13]:
market.select_dtypes(exclude='object').isnull().sum()

nu_meses_rescencia             45276
vl_faturamento_estimado_aux    27513
qt_filiais                         0
dtype: int64

In [14]:
#função que preenche o valores nulos com a variável categórica que mais aparece
def fill_nan_col(df_col):
    return df_col.fillna(df_col.value_counts().index[0])

In [15]:
#tratando os dados Nan categóricos
for i in market.select_dtypes(include='object').columns:
    market[i]= fill_nan_col(market[i])

In [16]:
#conferindo se funcionou!
market.select_dtypes(include='object').isnull().sum()
#como não há itens faltandes no id não precisamos nos preocupar

id                                     0
sg_uf                                  0
natureza_juridica_macro                0
de_ramo                                0
idade_emp_cat                          0
nm_divisao                             0
nm_segmento                            0
sg_uf_matriz                           0
de_saude_tributaria                    0
de_saude_rescencia                     0
de_nivel_atividade                     0
nm_meso_regiao                         0
nm_micro_regiao                        0
de_faixa_faturamento_estimado          0
de_faixa_faturamento_estimado_grupo    0
dtype: int64

In [17]:
#Tratando os dados não categóricos
for i in market.select_dtypes(exclude='object').columns:
    market[i]= fill_nan_col(market[i])

In [18]:
market.select_dtypes(exclude='object').isnull().sum()

nu_meses_rescencia             0
vl_faturamento_estimado_aux    0
qt_filiais                     0
dtype: int64

In [19]:
market.shape

(462298, 18)

In [20]:
market.shape

(462298, 18)

In [21]:
market.head()

,id,sg_uf,natureza_juridica_macro,de_ramo,idade_emp_cat,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,10 a 15,CONSTRUCAO DE EDIFICIOS,CONSTRUCAO,RN,VERDE,ACIMA DE 1 ANO,23.0,ALTA,LESTE POTIGUAR,NATAL,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",3132172.8,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,1 a 5,ALIMENTACAO,ALOJAMENTO E ALIMENTACAO,PI,CINZA,ATE 1 ANO,9.0,BAIXA,CENTRO NORTE PIAUIENSE,TERESINA,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",5 a 10,TRANSPORTE TERRESTRE,TRANSPORTE ARMAZENAGEM E CORREIO,AM,AMARELO,ACIMA DE 1 ANO,26.0,MEDIA,CENTRO AMAZONENSE,MANAUS,"ATE R$ 81.000,00","ATE R$ 81.000,00",50000.0,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,AM,OUTROS,SERVICOS DIVERSOS,5 a 10,REPARACAO E MANUTENCAO DE EQUIPAMENTOS DE INFO...,OUTRAS ATIVIDADES DE SERVICOS,AM,AMARELO,ACIMA DE 1 ANO,26.0,MEDIA,CENTRO AMAZONENSE,MANAUS,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,RN,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",1 a 5,SERVICOS DE ARQUITETURA E ENGENHARIA TESTES E ...,ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,RN,VERDE,ACIMA DE 1 ANO,27.0,ALTA,LESTE POTIGUAR,NATAL,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,0


In [22]:
market.to_csv('market_processed.csv', index=False)